# Libraries

In [32]:
import pandas as pd
import pyodbc
import os
from datetime import datetime
from openpyxl import load_workbook
import shutil
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

# Parameters

In [33]:
year = 2021
week = 27

# La ruta donde está la raw data
path_raw_data = r'C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Copservir\Copservir Faltante\{0}\S {1}\{0}_{1}.xlsx'
path_raw_data = path_raw_data.format(year, str(week).zfill(2))

# La ruta donde están las maestras
path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\COL\Copservir'

# La ruta donde guardaré el layout No B2B de ISV
path_original_layout = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\COL\ISV\No B2B\Data'

# Import Data

In [34]:
%%time
# Importación de la data semanal correspondiente
df = pd.read_excel(path_raw_data.format(year, str(week).zfill(2)))

# Importación de la maestra de locales
df_pos = pd.read_excel(path + '\\Maestra Locales Copservir.xlsx')

# Importación de la maestra de precios y productos
df_prices = pd.read_excel(path + '\\Maestra Precios Copservir.xlsx')

Wall time: 37.5 s


# Connection to DWH

In [35]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

qday = '''SELECT SemFin FROM Gnm_MasterOp.dbo.CatSemanas WHERE SemAnio = {0} AND SemNumero = {1};'''

In [36]:
dday = pd.read_sql(qday.format(year, week), conn)

# Clean Data & Merge

Se crea un tabla dinámica y un par de columnas, además se renombran algunas columnas.

In [37]:
df = df.pivot_table(
    index = ['codHoml', 'Código producto'], 
    values = ['Cantidad Saldo', 'so'], 
    aggfunc = 'sum'
).reset_index()

df['Fecha_Venta'] = dday.loc[0, 'SemFin']
df['Cadena'] = 'Copservir'

In [38]:
def unique_values(df, column):
    data = df.copy()
    vals_duplic = data[column].value_counts()[data[column].value_counts() > 1].index.tolist()
    data_unique_values = data[~data[column].isin(vals_duplic)].copy()
    for v in vals_duplic:
        data_unique_values = pd.concat([data_unique_values, data[data[column] == v][:1]], axis=0)
    data_unique_values.reset_index(drop=True, inplace=True)
    return data_unique_values

In [39]:
data_pos = unique_values(df_pos, 'PDV')

In [41]:
final = pd.merge(df, data_pos, left_on='codHoml', right_on='PDV', how='left')
final = final.merge(df_prices, left_on='Código producto', right_on='Cod_Hom', how='left')
df.shape[0] == final.shape[0]

True

In [42]:
final.isnull().sum()

codHoml            0
Código producto    0
Cantidad Saldo     0
so                 0
Fecha_Venta        0
Cadena             0
PDV                0
NOMBRE PDV         0
Cod_Hom            0
Cod_Prod           0
EAN                0
Descripcion        0
Precio             0
dtype: int64

In [43]:
# Creamos 2 listas de las columnas de Sell Out e Inventario para exportarlos
cols_sellout = ['Fecha_Venta', 'Cadena', 'Código producto', 'EAN', 'Descripcion', 'PDV', 'NOMBRE PDV', 'so']

cols_stock = ['Fecha_Venta', 'Cadena', 'Código producto', 'EAN', 'Descripcion', 'PDV', 'NOMBRE PDV', 'Cantidad Saldo']

# Export data

In [ ]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

Creando un nuevo folder donde se guardará el archivo ISV No B2B con la información de Copservir

In [ ]:
new_folder = path_original_layout + '\\{0}\\{1}'.format(year, str(week).zfill(2))

if not os.path.exists(new_folder):
    os.mkdir(new_folder)

Creando la ruta de donde se encuentra el layout de ISV y donde vamos a exportar la información

In [ ]:
path_original_layout = path_original_layout + '\\Formato No B2B.xlsx'
filename = 'Formato No B2B Copservir {1} - {0}.xlsx'.format(year, str(week).zfill(2))
path_export = new_folder + '\\' + filename

Exportación tanto de sell out como inventario:

In [ ]:
%%time
shutil.copy(path_original_layout, path_export)
write_layout(path_export, final[cols_sellout], True)
write_layout(path_export, final[cols_stock], False)